# Causal Inference with CEM and Weighted Regression

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from cem import match
from cem import coarsen
from cem.imbalance import L1
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
uri = os.environ["MONGODB_URI"]
client = MongoClient(uri, server_api=ServerApi("1"))
client.admin.command("ping")
db = client["real-estate"]
collection = db["listings"]

In [ ]:
since = datetime.now() - timedelta(days=30)

pipeline = [
        {
            "$match": {"rental": True, "datetime": {"$gte": since}, "bed": {"$lte": 4}},
        },
    ]

results = collection.aggregate(pipeline)

df = pd.DataFrame.from_records(results).set_index("_id")
df.head()

In [ ]:
df.describe()

In [ ]:
df[["bed", "bath", "parking"]].cov()

In [ ]:
df["council"].value_counts()

In [ ]:
councils = {
    "city of playford": "north",
    "city of port adelaide enfield": "north",
    "city of charles sturt": "west",
    "city of salisbury": "north",
    "corporation of the city of adelaide": "inner",
    "corporation of the city of unley": "inner",
    "city of marion": "south",
    "city of onkaparinga": "south",
    "city of campelltown": "east",
    "city of norwood payneham & st peters": "inner",
    "city of west torrens": "west",
    "city of tea tree gully": "north",
    "city of burnside": "east",
    "city of holdfast bay": "west",
    "mount barker district council": "hills",
    "city of prospect & city of port adelaide enfield": "north",
    "corporation of the town of walkerville": "inner",
    "city of mitcham": "south",
    "city of prospect": "inner",
    "adelaide hills council": "hills",
    "town of gawler": "outer",
    "the barossa council": "outer",
    "city of port adelaide enfield & city of tea tree gully": "north"
}

In [ ]:
y = df["price"]
X = df.drop(columns="price")[["bed", "bath", "parking", "council", "dwelling"]]

In [ ]:
# no matching
L1(X, "bed")
# very imbalanced

In [ ]:
# exact matching
# throw away examples from strata (defined by council, dwelling, bath and parking) that do not contain all levels of the treatment (number of beds)
weights = match(X, "bed")

print(f"{(weights > 0).sum()} observations remain. Threw away {(weights == 0).sum()}")

L1(X, "bed", weights)
# no examples left..

In [ ]:
# coarsened exact matching
# throw away examples from strata (defined by COARSENED council, dwelling, bath and parking) that do not contain all levels of the treatment (number of beds)
X_coarse = X.copy()

X_coarse["council"] = X_coarse["council"].map(councils)  # north, south, east, west, inner, hills, outer
X_coarse["parking"] = X_coarse["parking"] > 0  # yes or no
X_coarse["bath"] = pd.cut(X_coarse["bath"], bins=[-1, 1, 2, 100])  # 1, 2, 3+

weights = match(X_coarse, "bed")
X_coarse = X_coarse.loc[weights.index]  # not necessary, but it supresses that warning

print(f"{(weights > 0).sum()} observations remain. Threw away {(weights == 0).sum()}")

L1(X_coarse, "bed", weights)

In [ ]:
X_coarse[weights > 0].sort_values(["council", "dwelling", "bath", "parking", "bed"]).head(20)

In [ ]:
print(X_coarse[weights > 0]["dwelling"].value_counts())
print(X_coarse[weights > 0]["council"].value_counts())
print(X_coarse[weights > 0]["parking"].value_counts())
print(X_coarse[weights > 0]["bath"].value_counts())

In [ ]:
# after coarsened matching, there is very little imbalance, so i'm happy not to control for council and dwelling
model = sm.WLS(y, sm.add_constant(X[["bed"]]), weights=weights, hasconst=True)
model.exog_names[:] = ["constant", "bed"]
results = model.fit()

In [ ]:
summary = results.summary()
summary

In [ ]:
print(summary.as_html())

In [ ]:
# analysis of residuals